# Convolution Neural Network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

---------------------------------------------------------

## 1. Data (mnist)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("D:/2_Edu/tmp/data/", one_hot=True)

Extracting D:/2_Edu/tmp/data/train-images-idx3-ubyte.gz
Extracting D:/2_Edu/tmp/data/train-labels-idx1-ubyte.gz
Extracting D:/2_Edu/tmp/data/t10k-images-idx3-ubyte.gz
Extracting D:/2_Edu/tmp/data/t10k-labels-idx1-ubyte.gz


------------------------------------------------------------------------
## 2. CNN

### 2-1. Hyper-parameter

In [3]:
training_iters  = 200000
batch_size      = 100
learning_rate   = 0.001
display_step    = 1

### 2-2. Layer Parameter

In [4]:
n_input   = 28*28
n_classes = 10
dropout   = 0.75

### 2-3. Input

In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

keep_prob = tf.placeholder(tf.float32) ## for dropout

### 2-4. Modeling

    -----------------------------------------------------------------------------------
    Parameter
    -----------------------------------------------------------------------------------
    - intput: 28*28*1 << 1: feature number
    - filter: 3*3*1, 32ea
    - output: 32ea
    >> w = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01)) << stddev: random
    -----------------------------------------------------------------------------------
    - strides = [1,n,n,1] << 1: default setting
    - padding = 'SAME'    << input 28*28 -> output 28*28 (input-output same size)
    -----------------------------------------------------------------------------------
    - max pooling 2*2 이면 input의 반이 output size
    -----------------------------------------------------------------------------------


In [6]:
def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1,strides,strides,1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1,k,k,1], padding = 'SAME')


def conv_net(x, weights, biases, dropout):
    """Layering
    -----------------------------------------------------------------------------------
    input size | 전체 사이즈를 28*28*1로 처리
    -----------------------------------------------------------------------------------
    conv1      | filter 5*5*1,  32ea >> n*28*28*32
               | stride 2            >> n*14*14*32
    conv2      | filter 5*5*32, 64ea >> n*14*14*64
               | stride 2            >> n*7*7*64
    fc1        | reshape             >> 7*7*64, m   >> for fully-connected
    out        | classes number      >> m, 10
    -----------------------------------------------------------------------------------
    """

    x = tf.reshape(x, shape = [-1, 28, 28, 1]) ## -1: 전체 갯수(all)

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])     ## shape=(?, 28, 28, 32)
    conv1 = maxpool2d(conv1, k = 2)                      ## shape=(?, 14, 14, 32)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2']) ## shape=(?, 14, 14, 64)
    conv2 = maxpool2d(conv2, k = 2)                      ## shape=(?, 7, 7, 64)

    fc1   = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])  ## shape=(?, 3136)
    fc1   = tf.nn.relu(tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])) ## shape=(?, 1024)
    fc1   = tf.nn.dropout(fc1, dropout)                                       ## shape=(?, 1024)

    out   = tf.add(tf.matmul(fc1, weights['out']), biases['out']) ## shape=(?, 10)

    return out

In [7]:
weights = {'wc1':tf.Variable(tf.random_normal([5, 5,  1, 32], stddev = 0.01)),
           'wc2':tf.Variable(tf.random_normal([5, 5, 32, 64], stddev = 0.01)),
           'wd1':tf.Variable(tf.random_normal([7*7*64, 1024], stddev = 0.01)),
           'out':tf.Variable(tf.random_normal([1024, n_classes], stddev = 0.01))}

biases  = {'bc1':tf.Variable(tf.random_normal([32])),
           'bc2':tf.Variable(tf.random_normal([64])),
           'bd1':tf.Variable(tf.random_normal([1024])),
           'out':tf.Variable(tf.random_normal([n_classes]))}


model = conv_net(x, weights, biases, keep_prob)
print(model)

Tensor("Add_1:0", shape=(?, 10), dtype=float32)


### 2-5. Optimization

In [8]:
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 2-6. Test Accuracy

In [9]:
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy      = tf.reduce_mean(tf.cast(correct_model, tf.float32))

### 2-7. tf.Session

In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy],
                                 feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            print("Iter " + str(step*batch_size) + \
                  ", Minibatch Loss= " + "{:.6f}".format(loss) +  \
                  ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    print("==========================================================")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 100, Minibatch Loss= 2.616705, Training Accuracy= 0.11000
Iter 200, Minibatch Loss= 2.642316, Training Accuracy= 0.15000
Iter 300, Minibatch Loss= 2.558506, Training Accuracy= 0.14000
Iter 400, Minibatch Loss= 2.379109, Training Accuracy= 0.13000
Iter 500, Minibatch Loss= 2.340569, Training Accuracy= 0.13000
Iter 600, Minibatch Loss= 2.289788, Training Accuracy= 0.10000
Iter 700, Minibatch Loss= 2.350492, Training Accuracy= 0.12000
Iter 800, Minibatch Loss= 2.347479, Training Accuracy= 0.11000
Iter 900, Minibatch Loss= 2.372990, Training Accuracy= 0.12000
Iter 1000, Minibatch Loss= 2.362260, Training Accuracy= 0.11000
Iter 1100, Minibatch Loss= 2.353224, Training Accuracy= 0.11000
Iter 1200, Minibatch Loss= 2.328209, Training Accuracy= 0.13000
Iter 1300, Minibatch Loss= 2.337217, Training Accuracy= 0.11000
Iter 1400, Minibatch Loss= 2.321326, Training Accuracy= 0.12000
Iter 1500, Minibatch Loss= 2.320499, Training Accuracy= 0.11000
Iter 1600, Minibatch Loss= 2.313568, Training Acc

-------------------------------------------------------------------------
*End of Code*